In [1]:
import tensorflow as tf

print('Tensorflow Version:', tf.__version__)
print('Num GPUs Available:', len(tf.config.list_physical_devices('GPU')))

!nvidia-smi

Tensorflow Version: 2.9.2
Num GPUs Available: 1
Fri Dec  9 01:02:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    30W /  70W |      3MiB / 15109MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                        

In [2]:
from google.colab import drive 
import sys

drive.mount('/content/drive', force_remount=True) 

## This is the file path for TAs. Students will have to make a copy and then put in the file path themselves.
colab_path="/content/drive/My Drive/DeepLearning/2470project/model"
sys.path.append(colab_path)

Mounted at /content/drive


In [3]:
from preprocess import load_data

In [4]:
x_train, y_train = load_data([i for i in range(1, 16)], True)
x_test, y_test = load_data([1])

In [5]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(88, 4931392) (128, 4931392)
(88, 127808) (128, 127808)


In [6]:
x_train = x_train.T
y_train = y_train.T
x_test = x_test.T
y_test = y_test.T

In [7]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(1024, activation='leaky_relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(512, activation='leaky_relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(128, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(5e-3), metrics=[
            tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", threshold=0.5), 
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall()
        ])

model.fit(
    x_train, y_train,
    epochs=50, 
    batch_size=100,
    validation_data=(x_test, y_test)
)

Epoch 1/50
49314/49314 [==============================] - 181s 4ms/step - loss: 0.0249 - binary_accuracy: 0.9751 - precision: 0.0275 - recall: 1.2210e-05 - val_loss: 0.0192 - val_binary_accuracy: 0.9808 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
49314/49314 [==============================] - 177s 4ms/step - loss: 0.0249 - binary_accuracy: 0.9751 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.0192 - val_binary_accuracy: 0.9808 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/50
49314/49314 [==============================] - 176s 4ms/step - loss: 0.0249 - binary_accuracy: 0.9751 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.0192 - val_binary_accuracy: 0.9808 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/50
49314/49314 [==============================] - 175s 4ms/step - loss: 0.0249 - binary_accuracy: 0.9751 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.0192 - val_binary_accuracy: 0.9808 - val_precision: 

KeyboardInterrupt: ignored

In [6]:
import numpy as np

x_train = np.reshape(x_train.T, (x_train.shape[1] // 64, 64, 88))
y_train = np.reshape(y_train.T, (y_train.shape[1] // 64, 64, 128))
x_test = np.reshape(x_test.T, (x_test.shape[1] // 64, 64, 88))
y_test = np.reshape(y_test.T, (y_test.shape[1] // 64, 64, 128))

In [9]:
import tensorflow as tf

number_units = 256
number_layers = 3
number_classes = 128

model = tf.keras.Sequential()
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(number_units, return_sequences = "True",kernel_initializer='normal', activation='tanh')))
model.add(tf.keras.layers.Dropout(0.2))
for i in range(number_layers - 1):
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(number_units,return_sequences = "True",kernel_initializer='normal', activation='tanh')))
    model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(number_classes, kernel_initializer='normal', activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(5e-3), metrics=[
            tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", threshold=0.5), 
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall()
        ])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 64, 512)          706560    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64, 512)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 64, 512)          1574912   
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64, 512)           0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 64, 512)          1574912   
 nal)                                                            
                                                        

In [8]:
model.fit(
    x_train, y_train,
    epochs=50, 
    batch_size=100,
    validation_data=(x_test, y_test)
)

Epoch 1/50
 25/771 [..............................] - ETA: 54s - loss: 0.0398 - binary_accuracy: 0.9530 - precision: 0.0231 - recall: 0.0221

KeyboardInterrupt: ignored

In [10]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(number_units, return_sequences = "True",kernel_initializer='normal', activation='tanh'))
model.add(tf.keras.layers.Dropout(0.2))
for i in range(number_layers - 1):
    model.add(tf.keras.layers.LSTM(number_units,return_sequences = "True",kernel_initializer='normal', activation='tanh'))
    model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(number_classes, kernel_initializer='normal', activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(5e-3), metrics=[
            tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", threshold=0.5), 
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall()
        ])

In [11]:
model.fit(
    x_train, y_train,
    epochs=50, 
    batch_size=100,
    validation_data=(x_test, y_test)
)

Epoch 1/50
771/771 [==============================] - 34s 31ms/step - loss: 0.0244 - binary_accuracy: 0.9743 - precision_1: 0.0240 - recall_1: 7.9374e-04 - val_loss: 0.0181 - val_binary_accuracy: 0.9808 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/50
771/771 [==============================] - 23s 30ms/step - loss: 0.0233 - binary_accuracy: 0.9751 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: 0.0181 - val_binary_accuracy: 0.9808 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/50
771/771 [==============================] - 24s 31ms/step - loss: 0.0232 - binary_accuracy: 0.9751 - precision_1: 0.2725 - recall_1: 5.7873e-06 - val_loss: 0.0181 - val_binary_accuracy: 0.9808 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 4/50
771/771 [==============================] - 24s 31ms/step - loss: 0.0229 - binary_accuracy: 0.9751 - precision_1: 0.4186 - recall_1: 0.0018 - val_loss: 0.0178 - val_binary_accuracy: 0.9808 - val_precision_1